In [33]:
from pandas import read_csv
#Load data set
look_back = 15
split = 735-200
series_influ_A_df = read_csv('../temp_data/influA_vietnam_last_10_days.csv', index_col=0, engine='python')
series_influ_A_df = series_influ_A_df.iloc[200:]
train_df = series_influ_A_df.iloc[0:split]
test_df = series_influ_A_df.iloc[split-look_back:]
# series_influ_A_df = series_influ_A_df["case"]

In [34]:
train_df.corr()

,Influenza A - All types of surveillance,temp,humidity,dew,precipcover,tempmax,tempmin,windspeed
Influenza A - All types of surveillance,1.000000,0.285393,-0.048620,0.239193,0.117446,0.284604,0.276200,0.133062
temp,0.285393,1.000000,0.183046,0.945632,0.342243,0.992313,0.989397,0.269972
humidity,-0.048620,0.183046,1.000000,0.491313,0.488136,0.110095,0.282139,0.097711
dew,0.239193,0.945632,0.491313,1.000000,0.466450,0.914078,0.969951,0.276626
precipcover,0.117446,0.342243,0.488136,0.466450,1.000000,0.299688,0.398868,0.131281
tempmax,0.284604,0.992313,0.110095,0.914078,0.299688,1.000000,0.966022,0.263099
tempmin,0.276200,0.989397,0.282139,0.969951,0.398868,0.966022,1.000000,0.277953
windspeed,0.133062,0.269972,0.097711,0.276626,0.131281,0.263099,0.277953,1.000000


In [35]:
series_influ_A_df.isna().sum()

Influenza A - All types of surveillance    1
temp                                       0
humidity                                   0
dew                                        0
precipcover                                0
tempmax                                    0
tempmin                                    0
windspeed                                  0
dtype: int64

In [36]:
series_influ_A_df = series_influ_A_df.rename(columns= {"Influenza A - All types of surveillance": "case"})
series_influ_A_df = series_influ_A_df[["case", "temp", "tempmax", "dew","windspeed"]]

In [37]:
def exponential_moving_average(data, span):
    return data.ewm(span=span, adjust=False).mean()

In [38]:
series_influ_A_df = series_influ_A_df.dropna()
span = 52  # Bạn có thể điều chỉnh độ dài span tùy ý
series_influ_A_df['case'] = exponential_moving_average(series_influ_A_df['case'], span)
series_influ_A_df = series_influ_A_df.astype('float32')
series_influ_A_df = series_influ_A_df.values
# normalize the dataset
from sklearn.preprocessing import MinMaxScaler, RobustScaler
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(series_influ_A_df)
# scaler = RobustScaler()
# dataset = scaler.fit_transform(series_influ_A_df)
# Create Training and Test

train = dataset[:split, :]
test = dataset[split-look_back:, :]

import numpy as np
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

In [39]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [40]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import tensorflow as tf

from keras.losses import mean_squared_error

def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(mean_squared_error(y_true, y_pred))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(64, input_shape=(look_back, 5)))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss=root_mean_squared_error, optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100


c:\Users\nghiemxuan\Desktop\nghiem\GR3\GR3\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


520/520 - 2s - 4ms/step - loss: 0.1529
Epoch 2/100
520/520 - 1s - 2ms/step - loss: 0.1041
Epoch 3/100
520/520 - 1s - 2ms/step - loss: 0.0978
Epoch 4/100
520/520 - 1s - 2ms/step - loss: 0.0932
Epoch 5/100
520/520 - 1s - 2ms/step - loss: 0.0902
Epoch 6/100
520/520 - 1s - 2ms/step - loss: 0.0878
Epoch 7/100
520/520 - 1s - 2ms/step - loss: 0.0880
Epoch 8/100
520/520 - 1s - 2ms/step - loss: 0.0853
Epoch 9/100
520/520 - 1s - 2ms/step - loss: 0.0837
Epoch 10/100
520/520 - 1s - 2ms/step - loss: 0.0832
Epoch 11/100
520/520 - 1s - 2ms/step - loss: 0.0822
Epoch 12/100
520/520 - 1s - 2ms/step - loss: 0.0815
Epoch 13/100
520/520 - 1s - 2ms/step - loss: 0.0797
Epoch 14/100
520/520 - 1s - 2ms/step - loss: 0.0798
Epoch 15/100
520/520 - 1s - 2ms/step - loss: 0.0791
Epoch 16/100
520/520 - 1s - 2ms/step - loss: 0.0780
Epoch 17/100
520/520 - 1s - 2ms/step - loss: 0.0770
Epoch 18/100
520/520 - 1s - 2ms/step - loss: 0.0769
Epoch 19/100
520/520 - 1s - 2ms/step - loss: 0.0762
Epoch 20/100
520/520 - 1s - 2ms/s

In [41]:
model.save("../model/LSTM_4_4_ema_sigmoid.keras")